# NeRF: Representing Scenes as Neural Radiance Fields for View Synthesis

참고링크
+ blog
    + https://nuggy875.tistory.com/168
    + https://mvje.tistory.com/158
+ paper : https://arxiv.org/abs/2003.08934

## Contribution
Neural Radiance Fields(NeRF)를 제안, 이를 효과적으로 최적화하여 복잡한 입체적인 물체의 novel view를 생성해낸다.  
(SOTA찍음)

<img src="https://drive.google.com/uc?id=1nlyyJ90zOLeTmKhzJFcWP4WXR0K4HL-0
" heigth=200>

## Method


논문은 3가지 흐름으로 이야기한다.
1. NeRF Scene Representation : 공간을 어떻게 표현했는가
2. Volume Rendering : 해당 공간 정보를 이용해 2D이미지 투시로 어떻게 재구성해낼 것인가
3. Optimizing NeRF : 성능을 위한 추가작업
    + Positional encoding
    + Hierarchical volume sampling

<img src="https://drive.google.com/uc?id=1BcKb48KWzJaOUJCSTaLojOPXk9VK4E1q" height=250>

### NeRF Scene Representation

+ $\mathbf{x} = (x, y, z)$ : 3D상의 절대적 위치좌표
+ $\mathbf{d} = (\theta, \phi)$ : 2D viewing direction
+ $\mathbf{c} = RGB$ : 해당 방향에서 바라봤을 때, 드러나는 색상
+ $\sigma$ : volume density
+ $F_{\Theta} : (\mathbf{x}, \mathbf{d}) \rightarrow (\mathbf{c}, \sigma)$ : mapping network

__Goal__   
특정한 좌표 $\mathbf{x}$를 $\mathbf{d}$라는 각도에서 바라보았을 때, 이에 해당하는 색상과 밀도를 반환해내는 모델(MLP)를 학습하는 것이다.

<img src="https://drive.google.com/uc?id=15mXmbvIyugWrWXJt9rrDYEzXxkAYk4Rm" height=300>

+ 각 mlp마다 ReLU가 들어감.
+ 초록색은 input 값
+ "+" 표시는 concat
+ 붉은 색은 output 값

### Volume Rendering with Radiance Fields

+ $\mathbf{r}(t) = \mathbf{o} + t\mathbf{d}$ : $\mathbf{d}$ 시점에서 바라본 camera ray의 직선의 방정식
+ $C(\mathbf{r})$ : 해당 ray에서 보일 것으로 예상되는 color값

$C(\mathbf{r}) = \int^{t_f}_{t_n} T(t)\sigma(\mathbf{r}(t)\mathbf{c}(\mathbf{r}(t), \mathbf{d}))dt$,  
where $T(t) = \exp\bigg(-\int^t_{t_n}\sigma(\mathbf{r}(s))ds \bigg)$

C(r)의 의미 : 해당 ray에서 바라본 color 값을 어떻게 예측하느냐??   
주어진 직선 r에 대해서, 물체의 near/far bound인 $t_n, t_f$까지의   
밀도 x color를 적분해준다.  
이 때, T라는 투과율(Transmittance) 값을 곱해주어 멀 수록 희미해지도록한다.

but, 컴퓨터가 계산하기 위해서는 discrete하게 바꿔줘야한다. => quadrature(구적법)

간단하게, $[t_n, t_f]$의 구간을 $N$등분 한 점에 대해서 합해준다.

$\hat{C}(\mathbf{r}) = \sum^N_{i=1}T_i(1-\exp(-\sigma_i\delta_i))\mathbf{c}_i$,
where $T_i = \exp\bigg( -\sum^{i-1}_{j=1}\sigma_j\delta_j \bigg), \delta_i = t_{i+1}-t_i$

이 때, $\sigma\delta$가 아니라, $1-\exp(-\sigma\delta)$를 곱해주는 건 수치적분적인 방법론이라고 한다.  
(좀더 smoothing하게 해준다??)

### Optimizing a Neural Radiance Field

위의 방법은 분명 좋은 방법이지만... SOTA는 찍지 못했다고한다. 따라서, 두가지 추가적인 기법을 사용한다.

#### Positional encoding

MLP의 입력값인 $\mathbf{x}, \mathbf{d}$를 higher dimensional space로 mapping해주면, model이 마찬가지로 high frequency한 variation에 대해 더 잘 data를 fitting해준다는 굉장히 reasonable한 생각으로 적용한 것이다.

$\gamma : \mathbb{R} \rightarrow \mathbb{R}^{2L}$  
$\gamma(p) = (\sin(2^0\pi p), \cos(2^0\pi p), ..., \sin(2^{L-1}\pi p), \cos(2^{L-1}\pi p),)$

$\mathbf{x}$ : [-1,1]으로 normalize.L = 10   
$\mathbf{d}$ : unit vector를 사용. L = 4

다시 mlp를 보면...  
<img src="https://drive.google.com/uc?id=15mXmbvIyugWrWXJt9rrDYEzXxkAYk4Rm" height=300>

transformer에서도 마찬가지로 positional encoding을 통해   
이산적인 token들을 sequential하게 표현하기 위해서 적용한다.

하지만, 그와 다른 점은   
연속적인 input좌표를 high demensional space로 mapping해서 MLP가 더 high frequency function이 되도록 해주는 역할이다.

#### Hierarchical volume sampling

기존에 volume rendering하는 방법은 coarse한 color에 대해서 표현이 좋지만, fine한 부분에 대해서는 부족했다.

따라서 두가지 network를 최적화한다.

먼저, coarse는 앞에서 구한 것과 같다.  
$\hat{C}_c(\mathbf{r}) = \sum^{N_c}_{i=1}w_i\mathbf{c}_i$,
where $w_i = T_i(1-\exp(-\sigma_i\delta_i))$

이제, 각 weight들을 $\hat{w_i} = w_i / \sum^{N_c}_{j=1}w_j$해주어 확률밀도함수형태로 normalizing해준다.

그리고, 같은 ray에 대해서 위에서 구한 확률에 기반해서 $N_f$라는 fine locations set을 다시 셈플링해준다. 그리고 이를 통해 $\hat{C}_f(\mathbf{r})$을 계산해준다. 다만, 전체 color를 보존해주기 위해 $N_c+N_f$에 대해 계산해준다.

$\hat{C}_f(\mathbf{r}) = \sum^{N_c+N_f}_{i=1}w_i\mathbf{c}_i$,
where $w_i = T_i(1-\exp(-\sigma_i\delta_i))$

최종적으로 optimiazation하기 위한 Loss term은 다음과 같다.

$\mathcal{L} = \sum_{r\in\mathcal{R}}\bigg[||\hat{C}_c(\mathbf{r}) - C(\mathbf{r})||^2_2 || + \hat{C}_f(\mathbf{r}) - C(\mathbf{r})||^2_2 \bigg]$

$N_c=64, N_f=128$을 사용하였다.

## Experiment

<img src="https://drive.google.com/uc?id=1wFYFJ2KCTo06qCGJBTzg7ZIZBgjNLM4s" height=150>

<img src="https://drive.google.com/uc?id=15RYd96-Oz6j27yuVtFGZAuB_8VnNdRkt" height=200>



---

